In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import datetime
from math import sin, cos, sqrt, atan2, radians

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
RANDOM_SEED = 6    # Set a random seed for reproducibility!
from xgboost import XGBClassifier


In [0]:
#LOAD DATA
train_df = pd.read_csv("train.csv", index_col="tripid")
test_df = pd.read_csv( "test.csv", index_col="tripid")
submission_df = pd.read_csv("sample_submission.csv", index_col="tripid")

In [0]:
def train_label_to_num(df):
    if df['label'] == 'correct':
        return 1
    elif df['label'] == 'incorrect':
        return 0
train_df['label']=train_df.apply(train_label_to_num,axis=1)

In [0]:
# def pickupHourGroup(val):
#   val=int(val)
#   if(val<5 or val>22):
#     return 1
#   elif(4<val<8):
#     return 0
#   elif(7<val<11):
#     return 0
#   elif(10<val<17):
#     return 1
#   elif(16<val<21):
#     return 0
#   elif(20<val<23):
#     return 1


# get Distance FromLatLon
def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
  R = 6373.0
  lat1=radians(lat1)
  lat2=radians(lat2)
  lon1=radians(lon1)
  lon2=radians(lon2)
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
  c = 2 * atan2(sqrt(a), sqrt(1-a))
  distance = R * c
  return distance

In [0]:
def preProcess(train_df):
  train_df=train_df.dropna(how='any',axis=0)

  train_df['pickup_day'] =(train_df['pickup_time'].astype('datetime64')).dt.dayofweek
  train_df['drop_day'] =(train_df['drop_time'].astype('datetime64')).dt.dayofweek
  train_df['pickup_time'] = pd.to_datetime(train_df['pickup_time'])  
  train_df['drop_time'] = pd.to_datetime(train_df['drop_time'])  
  train_df['pickup_hour'] = train_df['pickup_time'].dt.strftime('%H')
  train_df['total_time']=train_df['duration']+train_df['meter_waiting_till_pickup']
  train_df['run_time']=train_df['duration']-train_df['meter_waiting']
  train_df['run_fare']=train_df['fare'] - train_df['meter_waiting_fare']-train_df['additional_fare']
  train_df['distance']=train_df.apply(lambda row : getDistanceFromLatLonInKm(row['pick_lat'], 
                      row['pick_lon'], row['drop_lat'], row['drop_lon']), axis = 1) 
  
  # train_df['drop_hour'] = train_df['drop_time'].dt.strftime('%H')
  # train_df['pickup_date'] = train_df['pickup_time'].dt.strftime('%D')
  # train_df['pickup_month'] = train_df['pickup_time'].dt.month
  # train_df['duration_cal']=(train_df['drop_time'] - train_df['pickup_time']).dt.total_seconds()
  # train_df["pickup_hour_group"]=train_df.apply(lambda row : pickupHourGroup(row['pickup_hour']), axis = 1)  

  # sl_holiday=[
  #              "1/15/2019","1/20/2019","2/4/2019","2/19/2019","3/4/2019","3/20/2019",
  #              "4/13/2019","4/14/2019","4/15/2019","4/19/2019","5/1/2019","5/18/2019","5/19/2019","5/20/2019",
  #              "6/5/2019","6/16/2019","7/16/2019","8/12/2019","8/14/2019",
  #              "9/13/2019","10/13/2019","10/27/2019",
  #              "11/10/2019","11/11/2019","11/12/2019","12/11/2019","12/25/2019",
  #              "01/10/2020","01/15/2020","02/8/2020","02/21/2020","03/9/2020","03/16/2020"]
  # sl_holidays = [str(pd.to_datetime(date)).split()[0] for date in sl_holiday]
  # train_df["Is_Holiday"]=[1 if str(val).split()[0] in sl_holidays else 0 for val in train_df['drop_time']]      


  train_df=train_df.drop(['pickup_time'], axis=1)
  train_df=train_df.drop(['drop_time'], axis=1)   
  return train_df         


In [0]:
train_df=preProcess(train_df)
features_df=train_df.drop(['label'], axis=1)
labels_df=train_df[['label']]

test_df=preProcess(test_df)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [0]:
fc=['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup',
    'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare', 'pickup_day', 'distance', 'total_time', 
    'run_time', 'pickup_hour', 'drop_day','run_fare']
# 0.9544642857142858
# 0.8378562322988795
features_df=features_df[fc]
test_df=test_df[fc]



In [0]:


# scaler = MinMaxScaler()
# # scaler = StandardScaler()
# features_df = features_df.iloc[:,:]
# features_df = scaler.fit_transform(features_df)
# features_df = scaler.transform(features_df)
# test_df = test_df.iloc[:,:]
# test_df = scaler.fit_transform(test_df)
# test_df = scaler.transform(test_df)
# print(features_df)

In [0]:
# def scalerFun(features_df):
#   scaler1 = MinMaxScaler()
#   # scaler1 = StandardScaler()
#   # features_df = features_df.iloc[:,:]
#   features_df = scaler1.fit_transform(features_df)
#   # features_df = scaler1.transform(features_df)

#   # kpca1 = KernelPCA(n_components=2, kernel = 'rbf')
#   # features_df = kpca1.fit_transform(features_df)
#   return features_df

# # features_df=scalerFun(features_df)
# # test_df=scalerFun(test_df)

In [0]:
estimators = MultiOutputClassifier(
 estimator=XGBClassifier(learning_rate =0.01,
 n_estimators=5000,
 max_depth=4,
 min_child_weight=6,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 reg_alpha=0.005,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
)

X_train, X_eval, y_train, y_eval = train_test_split(
    features_df,
    labels_df,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=RANDOM_SEED
)

In [0]:
%%time
full_pipeline = Pipeline([
    ("estimators", estimators),
])
# Train model
full_pipeline.fit(X_train, y_train)

preds = full_pipeline.predict(X_eval)

y_preds = pd.DataFrame(
    {
        "label": preds[:,0],
    },
    index = y_eval.index
)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_eval, y_preds))
from sklearn.metrics import f1_score
print(f1_score(y_eval, y_preds, average='macro'))

0.9480357142857143
0.8070253158486145
CPU times: user 1min 23s, sys: 2.46 s, total: 1min 26s
Wall time: 47.1 s


In [0]:
%%time 

full_pipeline.fit(features_df, labels_df)

None   # So we don't print out the whole pipeline representation

CPU times: user 1min 58s, sys: 2.67 s, total: 2min
Wall time: 1min 5s


In [0]:
test_probas = full_pipeline.predict(test_df)
submission_df = pd.read_csv("sample_submission.csv", 
                            index_col="tripid")
submission_df.head()

,prediction
tripid,
213284604,1
213286352,0
213293973,0
213294622,1
213298687,1


In [0]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["prediction"] = test_probas[:, 0]

In [0]:
submission_df.to_csv('submission.csv', index=True)

In [0]:
!head submission.csv

tripid,prediction
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1
